# Extracting features of 2020 movies from Wikipedia

In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Link

- Data can be found at [wiki](https://en.wikipedia.org/wiki/List_of_American_films_of_2020)

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
type(tables[0])

bs4.element.Tag

In [7]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [8]:
df = df1._append(df2._append(df3._append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [9]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,J A N U A R Y,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2],NaN
1,J A N U A R Y,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3],NaN
2,J A N U A R Y,10,Like a Boss,Paramount Pictures / Artists First,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4],NaN
3,J A N U A R Y,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN,NaN
4,J A N U A R Y,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5],NaN
...,...,...,...,...,...,...,...
268,D E C E M B E R,25,We Can Be Heroes,Netflix / Troublemaker Studios / Double R Prod...,Robert Rodriguez (director/screenplay); Priyan...,NaN,[239]
269,D E C E M B E R,25,News of the World,Universal Pictures / Perfect World Pictures,Paul Greengrass (director/screenplay); Luke Da...,NaN,[240]
270,D E C E M B E R,25,One Night in Miami...,Amazon Studios,Regina King (director); Kemp Powers (screenpla...,NaN,[241]
271,D E C E M B E R,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey M...,NaN,[242]


In [10]:
df_2020 = df[['Title','Cast and crew']]

In [11]:
df_2020

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...
1,Underwater,"William Eubank (director); Brian Duffield, Ada..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col..."
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...
...,...,...
268,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...
269,News of the World,Paul Greengrass (director/screenplay); Luke Da...
270,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...
271,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...


In [12]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '0e97b8b095304d243bf7e5f3cb4d9e3c'

In [13]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [14]:
df_2020.loc[:, ['genres']] = df_2020['Title'].map(lambda x: get_genre(str(x)))

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
df_2020

In [ ]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [ ]:
df_2020.loc[:, ['director_name']] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

In [ ]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [ ]:
df_2020.loc[:, ['actor_1_name']] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

In [ ]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [ ]:
df_2020.loc[:, ['actor_2_name']] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [ ]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [ ]:
df_2020.loc[:, ['actor_3_name']] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [ ]:
df_2020

In [ ]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [ ]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [ ]:
new_df20

In [ ]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [ ]:
new_df20.isna().sum()

In [ ]:
new_df20 = new_df20.dropna(how='any')

In [ ]:
new_df20.isna().sum()

In [ ]:
new_df20.loc[:, ['movie_title']] = new_df20['movie_title'].str.lower()

In [ ]:
new_df20

In [ ]:
old_df = pd.read_csv('final_wiki.csv')

In [ ]:
old_df

In [ ]:
final_df = old_df._append(new_df20,ignore_index=True)

In [ ]:
final_df

In [ ]:
final_df.to_csv('main_data.csv',index=False)